In [23]:
import os 
import sys
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from dataclasses import dataclass
from pathlib import Path
from cnnClassifier.constants import *
from cnnClassifier import logging,CustomException
from cnnClassifier.utils.common import read_yaml, create_directory

In [24]:
# os.chdir('d:\\codes\\DeepLearning_Proj\\proj1\\research')

In [25]:
os.chdir('../')
%pwd

'd:\\codes\\DeepLearning_Proj\\proj1'

In [26]:
@dataclass(frozen = True)
class PrepareCallbacksConfig:
    root_dir                    : Path
    tensorboard_root_log_dir    : Path
    checkpoint_model_filepath   : Path

In [27]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directory([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)

        create_directory([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir                   = Path(config.root_dir),
            tensorboard_root_log_dir   = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath  = Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [28]:
class PrepareCallback:
    def __init__(self, config : PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime('%y-%m-%d-%H-%M-%S')

        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f'tb_logs_at_{timestamp}'
        )

        return tf.keras.callbacks.TensorBoard(log_dir = tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only = True
        )

    # ckpt - checkpoint
    def get_tb_ckpt_callback(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

CREATING THE PIPELINE

In [29]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config(),
    prepare_callbacks = PrepareCallback(config = prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callback()

except Exception as e:
    CustomException(e,sys)

[2023-11-03 01:36:34,054] 31 root - INFO - YAML FILE config\config.yaml LOADED SUCCESSFULLY
[2023-11-03 01:36:34,057] 31 root - INFO - YAML FILE params.yaml LOADED SUCCESSFULLY
[2023-11-03 01:36:34,058] 50 root - INFO - CREATED DIRECTORY AT : artifacts
[2023-11-03 01:36:34,059] 50 root - INFO - CREATED DIRECTORY AT : artifacts\prepare_callbacks\checkpoint_dir
[2023-11-03 01:36:34,060] 50 root - INFO - CREATED DIRECTORY AT : artifacts\prepare_callbacks\tensorflow_log_dir
